In [1]:
# %pip install fastembed

In [2]:
from fastembed import TextEmbedding

## Q1. Embedding the query

In [3]:
embedding_model = TextEmbedding(
    model_name="jinaai/jina-embeddings-v2-small-en",
    model_type="text",
    device="cuda",
)

In [4]:
query = "I just discovered the course. Can I join now?"

In [5]:
embeddings_list = list(embedding_model.embed(query))
query_vector = embeddings_list[0]
query_vector.shape

(512,)

In [6]:
import numpy as np
np.min(query_vector)

np.float64(-0.11726373885183883)

In [7]:

np.linalg.norm(query_vector)

np.float64(1.0)

## Q2. Cosine similarity with another vector

In [8]:
doc = "Can I still join the course after the start date?"

embedding_generator = embedding_model.embed(doc)
doc_vector = next(embedding_generator)
np.min(doc_vector)

np.float64(-0.12396320482168117)

In [9]:
query_vector.dot(doc_vector)

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

In [10]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - Whast can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [11]:
doc_texts = [doc['text'] for doc in documents]
doc_embeddings = list(embedding_model.embed(doc_texts))
len(doc_embeddings)


5

In [12]:
embeddings_matrix = np.array(doc_embeddings)
embeddings_matrix.shape

(5, 512)

In [13]:
similarities = embeddings_matrix.dot(query_vector)
similarities

array([0.76296845, 0.81823782, 0.80853974, 0.71330788, 0.73044992])

In [14]:
np.argmax(similarities)

np.int64(1)

## Q4. Ranking by cosine, version two

In [15]:
doc_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
doc_embeddings = list(embedding_model.embed(doc_texts))
embeddings_matrix = np.array(doc_embeddings)
similarities = embeddings_matrix.dot(query_vector)

np.argmax(similarities)

np.int64(0)

## Q5. Selecting the embedding model

In [16]:
q5_model = TextEmbedding(model_name="BAAI/bge-small-en")
q5_model


In [17]:
q5_model.embedding_size

384

In [18]:
supported_models_info = TextEmbedding.list_supported_models()

supported_models_info


[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [19]:

min_model_name = None
min_model_dim = float('inf')
for model_info in supported_models_info:
   if model_info['dim'] < min_model_dim:
       min_model_dim = model_info['dim']
       min_model_name = model_info['model']

print(min_model_name)
print(min_model_dim)

BAAI/bge-small-en
384


## Q6. Indexing with qdrant (2 points)

In [20]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [21]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [22]:
from qdrant_client import QdrantClient, models


In [23]:
qd_client = QdrantClient("http://localhost:6333")

collection_name: str = "ml-zoomcamp-course-faqs"

if collection_name in [collection.name for collection in qd_client.get_collections().collections]:
    qd_client.delete_collection(collection_name)
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=min_model_dim,
        distance=models.Distance.COSINE
    )
)

True

In [24]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=min_model_name)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [25]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [26]:
q1 = "I just discovered the course. Can I join now?"

In [27]:
# embeddings_generator = q5_model.embed(q1)
# next(embeddings_generator)

In [28]:
query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=q1,
            model=min_model_name 
        ),
        limit=5,
        with_payload=True
    )
query_points.points[0].score

0.87031734